# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8588a33d60>
├── 'a' --> tensor([[ 0.6084, -0.7588,  0.5138],
│                   [ 0.3963,  0.5435, -0.3306]])
└── 'x' --> <FastTreeValue 0x7f8588a335e0>
    └── 'c' --> tensor([[-0.3289,  0.5932,  0.2774, -2.1261],
                        [-2.7271, -0.7969, -1.0272, -0.9531],
                        [ 0.4824, -1.2804, -1.0225, -1.6694]])

In [4]:
t.a

tensor([[ 0.6084, -0.7588,  0.5138],
        [ 0.3963,  0.5435, -0.3306]])

In [5]:
%timeit t.a

61.4 ns ± 0.0657 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8588a33d60>
├── 'a' --> tensor([[ 0.4994, -0.2943, -1.4299],
│                   [-2.2572, -1.6841, -1.2392]])
└── 'x' --> <FastTreeValue 0x7f8588a335e0>
    └── 'c' --> tensor([[-0.3289,  0.5932,  0.2774, -2.1261],
                        [-2.7271, -0.7969, -1.0272, -0.9531],
                        [ 0.4824, -1.2804, -1.0225, -1.6694]])

In [7]:
%timeit t.a = new_value

58.8 ns ± 0.0668 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6084, -0.7588,  0.5138],
               [ 0.3963,  0.5435, -0.3306]]),
    x: Batch(
           c: tensor([[-0.3289,  0.5932,  0.2774, -2.1261],
                      [-2.7271, -0.7969, -1.0272, -0.9531],
                      [ 0.4824, -1.2804, -1.0225, -1.6694]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6084, -0.7588,  0.5138],
        [ 0.3963,  0.5435, -0.3306]])

In [11]:
%timeit b.a

54.6 ns ± 0.0321 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.5708,  0.1319,  1.3915],
               [-0.1372, -0.7080, -1.2420]]),
    x: Batch(
           c: tensor([[-0.3289,  0.5932,  0.2774, -2.1261],
                      [-2.7271, -0.7969, -1.0272, -0.9531],
                      [ 0.4824, -1.2804, -1.0225, -1.6694]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.18 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

884 ns ± 7.06 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 35.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 336 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 307 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f84b9d14f40>
├── 'a' --> tensor([[[ 0.6084, -0.7588,  0.5138],
│                    [ 0.3963,  0.5435, -0.3306]],
│           
│                   [[ 0.6084, -0.7588,  0.5138],
│                    [ 0.3963,  0.5435, -0.3306]],
│           
│                   [[ 0.6084, -0.7588,  0.5138],
│                    [ 0.3963,  0.5435, -0.3306]],
│           
│                   [[ 0.6084, -0.7588,  0.5138],
│                    [ 0.3963,  0.5435, -0.3306]],
│           
│                   [[ 0.6084, -0.7588,  0.5138],
│                    [ 0.3963,  0.5435, -0.3306]],
│           
│                   [[ 0.6084, -0.7588,  0.5138],
│                    [ 0.3963,  0.5435, -0.3306]],
│           
│                   [[ 0.6084, -0.7588,  0.5138],
│                    [ 0.3963,  0.5435, -0.3306]],
│           
│                   [[ 0.6084, -0.7588,  0.5138],
│                    [ 0.3963,  0.5435, -0.3306]]])
└── 'x' --> <FastTreeValue 0x7f84b9d14f10>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 99 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f84c2dfa190>
├── 'a' --> tensor([[ 0.6084, -0.7588,  0.5138],
│                   [ 0.3963,  0.5435, -0.3306],
│                   [ 0.6084, -0.7588,  0.5138],
│                   [ 0.3963,  0.5435, -0.3306],
│                   [ 0.6084, -0.7588,  0.5138],
│                   [ 0.3963,  0.5435, -0.3306],
│                   [ 0.6084, -0.7588,  0.5138],
│                   [ 0.3963,  0.5435, -0.3306],
│                   [ 0.6084, -0.7588,  0.5138],
│                   [ 0.3963,  0.5435, -0.3306],
│                   [ 0.6084, -0.7588,  0.5138],
│                   [ 0.3963,  0.5435, -0.3306],
│                   [ 0.6084, -0.7588,  0.5138],
│                   [ 0.3963,  0.5435, -0.3306],
│                   [ 0.6084, -0.7588,  0.5138],
│                   [ 0.3963,  0.5435, -0.3306]])
└── 'x' --> <FastTreeValue 0x7f858aabb280>
    └── 'c' --> tensor([[-0.3289,  0.5932,  0.2774, -2.1261],
                        [-2.7271, -0.7969, -1.0272, -0.9531],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 56.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.8 µs ± 86.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.3289,  0.5932,  0.2774, -2.1261],
                       [-2.7271, -0.7969, -1.0272, -0.9531],
                       [ 0.4824, -1.2804, -1.0225, -1.6694]],
              
                      [[-0.3289,  0.5932,  0.2774, -2.1261],
                       [-2.7271, -0.7969, -1.0272, -0.9531],
                       [ 0.4824, -1.2804, -1.0225, -1.6694]],
              
                      [[-0.3289,  0.5932,  0.2774, -2.1261],
                       [-2.7271, -0.7969, -1.0272, -0.9531],
                       [ 0.4824, -1.2804, -1.0225, -1.6694]],
              
                      [[-0.3289,  0.5932,  0.2774, -2.1261],
                       [-2.7271, -0.7969, -1.0272, -0.9531],
                       [ 0.4824, -1.2804, -1.0225, -1.6694]],
              
                      [[-0.3289,  0.5932,  0.2774, -2.1261],
                       [-2.7271, -0.7969, -1.0272, -0.9531],
                       [ 0.4824, -1.2804, -1.0225, -1.6694]],

In [26]:
%timeit Batch.stack(batches)

78.1 µs ± 169 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.3289,  0.5932,  0.2774, -2.1261],
                      [-2.7271, -0.7969, -1.0272, -0.9531],
                      [ 0.4824, -1.2804, -1.0225, -1.6694],
                      [-0.3289,  0.5932,  0.2774, -2.1261],
                      [-2.7271, -0.7969, -1.0272, -0.9531],
                      [ 0.4824, -1.2804, -1.0225, -1.6694],
                      [-0.3289,  0.5932,  0.2774, -2.1261],
                      [-2.7271, -0.7969, -1.0272, -0.9531],
                      [ 0.4824, -1.2804, -1.0225, -1.6694],
                      [-0.3289,  0.5932,  0.2774, -2.1261],
                      [-2.7271, -0.7969, -1.0272, -0.9531],
                      [ 0.4824, -1.2804, -1.0225, -1.6694],
                      [-0.3289,  0.5932,  0.2774, -2.1261],
                      [-2.7271, -0.7969, -1.0272, -0.9531],
                      [ 0.4824, -1.2804, -1.0225, -1.6694],
                      [-0.3289,  0.5932,  0.2774, -2.1261],
                   

In [28]:
%timeit Batch.cat(batches)

146 µs ± 449 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

323 µs ± 2.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
